Set Parameters

In [ ]:
similarity_cutoff = 0.05 
#time_slice = slice("2021-09-12 14:00:00", "2021-09-12 14:00:00")
time_slice = slice("2021-09-12 14:00:00", "2021-09-12 22:00:00")

Import files 

In [ ]:
import xarray as xr 

In [ ]:
compound = xr.open_dataset('compound_nicholas.nc', chunks={'time': 'auto'})
surge = xr.open_dataset('surge_nicholas.nc', chunks={'time': 'auto'})
rivers = xr.open_dataset('rivers_nicholas.nc', chunks={'time':'auto'})

In [ ]:
x = compound['x']; y = compound['y'] 
ele_X_nodes = compound['element']-1 #extract element to global node map (connectivity map)

compound_zeta = compound['zeta'].sel(time=time_slice)
surge_zeta = surge['zeta'].sel(time=time_slice)
rivers_zeta = rivers['zeta'].sel(time=time_slice)

Calculate Differences, Compute Divergence Metric

In [14]:
from numba import vectorize, float64
import numpy as np 

#Need to formalize this logic. 
@vectorize([float64(float64,float64)])
def bathymetric_rel_diff(a: float, b: float) -> float:
    """
    Takes a relative-to-minuend difference between two bathymetries, making optimizations where dry cells are 
    encountered. Intended to be combined with Numba's vectorize and Xarray's apply_ufunc.                                                                                                                        

    Parameters
    ----------
    a : np.ndarray
        Minuend. 
    b : np.ndarray
        Subtrahend. 
    
    Returns
    -------
    difference : np.float
    
    Examples
    --------
    >>> a = [1 nan nan]
    >>> b = [1 1 nan]
    >>> xr.apply_ufunc(fast_bathymetric_diff_atomic,a,b)
    [0 -1 0]
    
    """    
    #Case where both are dry 
    if np.isnan(a) and np.isnan(b): 
        return 0
    
    #Double check this behavior. 
    if np.isnan(a): 
        return 999 
    if np.isnan(b): 
        b = 0 
    
    return abs((a-b)/a) 

#currently, this does not set any nan's, but maybe this is okay with holoviews. 
compound_surge_difference = xr.apply_ufunc(bathymetric_rel_diff, compound_zeta, surge_zeta, dask ="parallelized")
river_surge_difference = xr.apply_ufunc(bathymetric_rel_diff, compound_zeta, rivers_zeta, dask='parallelized')

compound_surge_difference_max = compound_surge_difference.max(dim="time")
river_surge_difference_max = river_surge_difference.max(dim="time")

divergence_of_surge_and_rivers = np.minimum(compound_surge_difference_max,river_surge_difference_max) 

Mesh time series data

In [ ]:
import geoviews as gv 
import holoviews as hv 
hv.extension("bokeh")
from holoviews import opts
from holoviews.operation.datashader import datashade,rasterize
import datashader as ds


datashade.precompute=True 
size = dict(width=800, height=600)
opts.defaults(
    opts.Image(**size), opts.RGB(**size), opts.VectorField(**size))

times = compound['time'].sel(time=time_slice).to_numpy() 

points = gv.operation.project_points(gv.Points((x,y)))

tris = ele_X_nodes.to_numpy()

def mesh_at_time(timestamp): 
    depth_points = points.add_dimension('zeta', 0, compound_surge_difference.sel(time=timestamp), vdim = True)
    return gv.TriMesh((tris, depth_points))

meshes_dict = {t:mesh_at_time(t) for t in times}
meshes_HoloMap = hv.HoloMap(meshes_dict, kdims="times")

tiles = gv.tile_sources.OSM 
#figure out how to set the range properly here (using inf not using 3)
meshes_raster = rasterize(meshes_HoloMap, interpolation='linear', aggregator=ds.mean('zeta')).opts(colorbar=True, clim=(0.01, 0.1), color_levels=9, clipping_colors={'min': 'transparent', 'max': 'green', 'NaN': 'transparent'})
plot = tiles * meshes_raster 


In [ ]:
hv.output(plot, holomap='scrubber', fps=4)

Mesh and plot divergence metric 

In [15]:
depth_points = points.add_dimension('zeta', 0, divergence_of_surge_and_rivers, vdim = True)
maxmesh = gv.TriMesh((tris, depth_points))
maxmesh_raster = rasterize(maxmesh, interpolation='linear', aggregator=ds.max('zeta')).opts(colorbar=True, 
                                                                                            clim=(0.02, 0.5), 
                                                                                            color_levels=9, 
                                                                                            clipping_colors={'min': 'transparent', 'max': 'green', 'NaN': 'transparent'})
plot = tiles * maxmesh_raster 
hv.output(plot)

BokehModel(combine_events=True, render_bundle={'docs_json': {'53f447e0-a0e7-4b82-8bef-87b2636a60b2': {'version…

Task exception was never retrieved
future: <Task finished name='Task-122' coro=<Callback.process_on_change() done, defined at /opt/miniconda3/envs/transition-zone/lib/python3.11/site-packages/holoviews/plotting/bokeh/callbacks.py:328> exception=UnsetValueError("figure(id='p1446', ...).inner_height doesn't have a value set")>
Traceback (most recent call last):
  File "/opt/miniconda3/envs/transition-zone/lib/python3.11/site-packages/holoviews/plotting/bokeh/callbacks.py", line 347, in process_on_change
    msg[attr] = self.resolve_attr_spec(path, cb_obj)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/transition-zone/lib/python3.11/site-packages/holoviews/plotting/bokeh/callbacks.py", line 254, in resolve_attr_spec
    resolved = getattr(resolved, p, None)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/transition-zone/lib/python3.11/site-packages/bokeh/core/property/descriptors.py", line 283, in __get__
    raise UnsetValueError(